In [65]:
import csv
import json
csvfile = open('labels.csv', 'r',encoding = "utf8")
jsonfile = open('lab.json', 'w')

fieldnames = ("id","occupation","gender","fame","birthyear")
reader = csv.DictReader( csvfile, fieldnames)
out = json.dumps( [ row for row in reader ] )
jsonfile.write(out)
            
            

csvfile = open('feed.csv', 'r',encoding = "utf8")
jsonfile = open('cel.json', 'w')

fieldnames = ("id","text")
reader = csv.DictReader( csvfile, fieldnames)
out = json.dumps( [ row for row in reader ] )
jsonfile.write(out)

In [ ]:

with open('cel.json',encoding='utf-8') as f:
        for line in f:
            data = json.loads(line)
tweets=data


with open('lab.json',encoding='utf-8') as f:
        for line in f:
            data1 = json.loads(line)
labels=data1


In [ ]:
def _preprocess_feed(tweet: str):
 
    t = tweet.lower()
    t = re.sub(url_re, " <URL> ", t)
    t = t.replace("\n", "")
    t = t.replace("#", " <HASHTAG> ")
    t = re.sub(mention_re, " <USER> ", t)
    t = re.sub(smile_re, " <EMOTICON> ", t)
    t = re.sub(emoji_re, " <EMOJI> ", t)
    t = re.sub(time_re, " <TIME> ", t)
    t = re.sub(numbers_re, " <NUMBER> ", t)
    t = re.sub(not_ascii_re, "", t)
    t = re.sub(space_collapse_re, " ", t)
    t = t.strip()
    return t

In [85]:
from bs4 import BeautifulSoup
import unicodedata
from nltk.tokenize import word_tokenize
import re

def untokenize(words):
    """Untokenizing a text undoes the tokenizing operation, restoring
    punctuation and spaces to the places that people expect them to be.
    Ideally, `untokenize(tokenize(text))` should be identical to `text`,
    except for line breaks.
    """
    text = ' '.join(words)
    step1 = text.replace("`` ", '"').replace(" ''", '"').replace('. . .', '...')
    step2 = step1.replace(" ( ", " (").replace(" ) ", ") ")
    step3 = re.sub(r' ([.,:;?!%]+)([ \'"`])', r"\1\2", step2)
    step4 = re.sub(r' ([.,:;?!%]+)$', r"\1", step3)
    step5 = step4.replace(" '", "'").replace(" n't", "n't").replace(
        "can not", "cannot")
    step6 = step5.replace(" ` ", " '")
    return step6.strip()


# https://stackoverflow.com/a/47091490
def decontracted(phrase):
    """Convert contractions like "can't" into "can not"
    """
    # specific
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    #phrase = re.sub(r"n't", " not", phrase) # resulted in "ca not" when sentence started with "can't"
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase


# https://github.com/rishabhverma17/sms_slang_translator/blob/master/slang.txt
slang_abbrev_dict = {
    'AFAIK': 'As Far As I Know',
    'AFK': 'Away From Keyboard',
    'ASAP': 'As Soon As Possible',
    'ATK': 'At The Keyboard',
    'ATM': 'At The Moment',
    'A3': 'Anytime, Anywhere, Anyplace',
    'BAK': 'Back At Keyboard',
    'BBL': 'Be Back Later',
    'BBS': 'Be Back Soon',
    'BFN': 'Bye For Now',
    'B4N': 'Bye For Now',
    'BRB': 'Be Right Back',
    'BRT': 'Be Right There',
    'BTW': 'By The Way',
    'B4': 'Before',
    'B4N': 'Bye For Now',
    'CU': 'See You',
    'CUL8R': 'See You Later',
    'CYA': 'See You',
    'FAQ': 'Frequently Asked Questions',
    'FC': 'Fingers Crossed',
    'FWIW': 'For What It\'s Worth',
    'FYI': 'For Your Information',
    'GAL': 'Get A Life',
    'GG': 'Good Game',
    'GN': 'Good Night',
    'GMTA': 'Great Minds Think Alike',
    'GR8': 'Great!',
    'G9': 'Genius',
    'IC': 'I See',
    'ICQ': 'I Seek you',
    'ILU': 'I Love You',
    'IMHO': 'In My Humble Opinion',
    'IMO': 'In My Opinion',
    'IOW': 'In Other Words',
    'IRL': 'In Real Life',
    'KISS': 'Keep It Simple, Stupid',
    'LDR': 'Long Distance Relationship',
    'LMAO': 'Laugh My Ass Off',
    'LOL': 'Laughing Out Loud',
    'LTNS': 'Long Time No See',
    'L8R': 'Later',
    'MTE': 'My Thoughts Exactly',
    'M8': 'Mate',
    'NRN': 'No Reply Necessary',
    'OIC': 'Oh I See',
    'OMG': 'Oh My God',
    'PITA': 'Pain In The Ass',
    'PRT': 'Party',
    'PRW': 'Parents Are Watching',
    'QPSA?': 'Que Pasa?',
    'ROFL': 'Rolling On The Floor Laughing',
    'ROFLOL': 'Rolling On The Floor Laughing Out Loud',
    'ROTFLMAO': 'Rolling On The Floor Laughing My Ass Off',
    'SK8': 'Skate',
    'STATS': 'Your sex and age',
    'ASL': 'Age, Sex, Location',
    'THX': 'Thank You',
    'TTFN': 'Ta-Ta For Now!',
    'TTYL': 'Talk To You Later',
    'U': 'You',
    'U2': 'You Too',
    'U4E': 'Yours For Ever',
    'WB': 'Welcome Back',
    'WTF': 'What The Fuck',
    'WTG': 'Way To Go!',
    'WUF': 'Where Are You From?',
    'W8': 'Wait',
    '7K': 'Sick:-D Laugher'
}


def unslang(text):
    """Converts text like "OMG" into "Oh my God"
    """
    if text.upper() in slang_abbrev_dict.keys():
        return slang_abbrev_dict[text.upper()]
    else:
        return text



# Reference : https://gist.github.com/slowkow/7a7f61f495e3dbb7e3d767f97bd7304b


def replace_urls(text):
    text=re.sub(r"http\S+", 'URL', text)
    text=re.sub(r"www\S+", 'URL', text)
    text=re.sub(r"pic.twitter.com\S+", 'URL', text)
    text=re.sub(r'https.*[^ ]', 'URL',text)
    
    return text

def clean(reg_exp, text):
    text = re.sub(reg_exp, " ", text)

    # replace multiple spaces with one.
    text = re.sub('\s{2,}', ' ', text)

    return text

def remove_accented_chars(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

def strip_html_tags(text):
    soup = BeautifulSoup(text, "html.parser")
    stripped_text = soup.get_text()
    return stripped_text

def clean_all(t):
    
    # first do bulk cleanup on tokens that don't depend on word tokenization

    # remove xml tags
    t = clean(r"<[^>]+>", str(t))
    t = clean(r"&lt;", str(t))
    t = clean(r"&gt;", str(t))

    # remove URLs
    t = replace_urls(t)

    
    
    # Removing HTML tags
    t=strip_html_tags(t)
    
    #remove_accented_chars
    t=remove_accented_chars(t)

    # https://stackoverflow.com/a/35041925
    # replace multiple punctuation with single. Ex: !?!?!? would become ?
    t = clean(r'[\?\.\!]+(?=[\?\.\!])', t)

    

    # expand common contractions like "I'm" "he'll"
    t = decontracted(t)

    # now remove/expand bad patterns per word
    words = word_tokenize(t)

    clean_words = []

    for w in words:
        # normalize punctuation
        w = re.sub(r'&', 'and', w)

        # expand slang like OMG = Oh my God
        w = unslang(w)


        clean_words.append(w)

    # join the words back into a full string
    t = untokenize(clean_words)


    # finally, remove any non ascii and special characters that made it through
    t = clean(r"[^A-Za-z0-9\.\'!\?,\$]", t)

    return t

In [90]:
tweet=[]
birthyear=[]
gender=[]
occupation=[]
ids=[]
fame=[]
for i in range(len(tweets)):
    if i!=0:
        tweet.append(clean_all(tweets[i]["text"]))
        birthyear.append(labels[i]['birthyear'])
        gender.append(labels[i]['gender'])
        occupation.append(labels[i]['occupation'])
        ids.append(labels[i]['id'])
        fame.append(labels[i]['fame'])

In [91]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from zipfile import ZipFile
import json

celebrity = {
    'id':ids,
    'text': tweet,
        'birthyear': birthyear,
         'gender':gender,
           'occupation':occupation,
    'fame':fame
        }


df = pd.DataFrame(celebrity)

In [92]:
df.head()

,id,text,birthyear,gender,occupation,fame
0,22704,Back at it with looking for .her Circa early 2...,C,female,performer,superstar
1,46305,The last presidential election turned on fewer...,D,male,politics,superstar
2,30260,Angels obbieWilliams t.co A6rx5WA0ieAmazing Th...,B,female,performer,superstar
3,4874,"Listen to Shallow, Always Remember Us This Way...",B,female,creator,superstar
4,41392,"So happy for my island! Vote for Madeira, for ...",B,male,manager,superstar


In [93]:
df.to_csv("celebrity_datas.csv")